# **06. AZURE GPT 데이터로 사용하기 위한 사전 작업**

##### **AZURE GPT 프로젝트 대상 아파트 범위**  
* 지역: 경기도 화성시, 평택시, 성남시, 과천시, 하남시, 광명시, 수원시
* 입주예정연도: 2023년 이후
* **2023년 이전 아파트들은 2023년 8월 기준 네이버 부동산 크롤링 데이터에 정보가 존재하여 청약 당시 공급액과 현재 시점 매매가를 비교할 수 있는 아파트만 사용**

In [1]:
import pandas as pd
apply_apartment = pd.read_csv("apartment_20230905.csv") # 청약 데이터
exist_apartment = pd.read_excel("부동산데이터_수도권_202308.xlsx") # 2023년 8월 기준 네이버부동산에 등록되어있는 아파트 데이터
exist_apartment = exist_apartment.copy()

In [2]:
# 프로젝트 대상 2023년 이후 입주 예정 아파트는 93개
ours = apply_apartment[(apply_apartment['기초'].isin(['화성시', '평택시', '성남시', '과천시', '하남시', '광명시', '수원시'])) & 
                       (apply_apartment['입주예정연도'] >= 2023)]
ours['아파트명'].nunique()

93

#### **2023년 이전 아파트들 중 청약 데이터와 현재 부동산 데이터에 동시에 존재하는 아파트 탐색**  
* 분양 당시 아파트명과 현재 아파트명이 달라졌거나 또는 각각의 데이터의 이름이 다른 동일 아파트를 찾는 것이 목표
* 찾야아하는 아파트는 아래와 같이 68개

In [3]:
excluded = apply_apartment[(apply_apartment['기초'].isin(['화성시', '평택시', '성남시', '과천시', '하남시', '광명시', '수원시'])) &
                           (apply_apartment['입주예정연도'] <= 2023)]
excluded['아파트명'].nunique() 

68

In [4]:
# 청약 데이터와 비슷한 컬럼으로 맞추기
exist_apartment.drop(columns=['영구임대세대수', '국민임대세대수', '민간임대세대수', '공공임대세대수', '장기임대세대수', '기타임대세대수'],
                     inplace=True)
exist_apartment['광역'] = exist_apartment['법정동주소'].apply(lambda x: x.split(" ")[0])
exist_apartment['기초'] = exist_apartment['법정동주소'].apply(lambda x: x.split(" ")[1])
exist_apartment['아파트명'] = exist_apartment['아파트명'].str.replace(" ", "")

In [5]:
exist = exist_apartment[exist_apartment['기초'].isin(['화성시', '평택시', '성남시', '과천시', '하남시', '광명시', '수원시'])]
exist.groupby('기초')['아파트명'].nunique()

기초
과천시     21
광명시    127
성남시    448
수원시    776
평택시    459
하남시    128
화성시    401
Name: 아파트명, dtype: int64

최초 
동시에 존재하는 아파트는 28개 

In [6]:
excluded_list = list(excluded['아파트명'].unique())
exist_list = list(exist['아파트명'].unique())
intersection = set(excluded_list) & set(exist_list) 
len(intersection)

28

In [7]:
# 비교해야 하는 아파트명
looking_for = list(set(excluded_list)-set(intersection)) 
looking_for_2 = list(set(exist_list)-set(intersection))

아래는 '제일'이 들어가는 아파트로 찾은 예시

In [8]:
result_1_제일 = [apartment for apartment in looking_for if '제일' in apartment]
result_1_제일

['고덕신도시A41BL제일풍경채2차', '고덕신도시A42BL제일풍경채3차센텀']

In [9]:
result_2_제일 = [apartment for apartment in looking_for_2 if '제일' in apartment]
result_2_제일

['고덕국제신도시제일풍경채',
 '제일하이빌1차',
 'SC제일행복(주상복합)',
 '동탄예당마을우미린제일풍경채',
 '제일풍경채2단지',
 '제일풍경채에듀앤파크',
 '무지개8단지제일',
 '고덕신도시제일풍경채센텀3차',
 '제일하이빌2차',
 '향남시범살구꽃마을제일오투그란데',
 '제일풍경채장당센트럴',
 '제일풍경채',
 '고덕신도시제일풍경채2차에듀',
 '제일풍경채1단지',
 '제일']

In [10]:
excluded.loc[excluded['아파트명'] == '고덕신도시A42BL제일풍경채3차센텀', '아파트명'] = '고덕신도시제일풍경채센텀3차'
excluded.loc[excluded['아파트명'] == '고덕신도시A41BL제일풍경채2차', '아파트명'] = '고덕신도시제일풍경채2차에듀'

**위와 같이 각각 비교한 아파트명을 통일시키는 작업**

In [11]:
excluded_change = [
                ('이안평택안중역아파트', '이안평택안중역'),
                ('평택고덕국제신도시A-43블럭호반써밋II', '호반써밋고덕국제신도시에듀파크'),
                ('더샵지제역센트럴파크1BL', '더샵지제역센트럴파크(1BL)'),
                ('더샵지제역센트럴파크2BL', '더샵지제역센트럴파크(2BL)'),
                ('서광교파크뷰', '서광교파크뷰(주상복합)'),
                ('위례신도시A3-10BL중흥S-클래스', '위례중흥S-클래스'),
                ('과천푸르지오어울림라비엔오', '과천푸르지오라비엔오'),
                ('영흥공원푸르지오파크비엔', '영흥숲푸르지오파크비엔'),
                ('매교역푸르지오SKVIEW(수원)', '매교역푸르지오SK뷰'),
                ('판교밸리자이1단지', '판교밸리자이1단지(주상복합)'),
                ('판교밸리자이2단지', '판교밸리자이2단지(주상복합)'),
                ('판교밸리자이3단지', '판교밸리자이3단지(주상복합)'),
                ('고덕신도시A42BL제일풍경채3차센텀', '고덕신도시제일풍경채센텀3차'),
                ('고덕신도시A41BL제일풍경채2차', '고덕신도시제일풍경채2차에듀'),
                ('만강아파트', '만강(주상복합)'),
                ('동탄역헤리엇', '동탄역헤리엇(주상복합)'),
                ('중앙하이츠금광프리미엄아파트', '중앙하이츠금광프리미엄'),
                ('화성태안3지구우미린센트포레B-1블럭', '화성태안3지구우미린센트포레B-1BL'),
                ('화성태안3지구우미린센트포레B-2블럭', '화성태안3지구우미린센트포레B-2BL'),
                ('위례신도시A3-2BL우미린2차', '위례우미린'), 
                ('화성남양뉴타운C-2BL리젠시빌란트', '화성시청역리젠시빌란트'),
                ('신동탄롯데캐슬나노시티', '나노시티역롯데캐슬'),
                ('영통롯데캐슬엘클래스1블록', '영통롯데캐슬엘클래스1BL'),
                ('영통롯데캐슬엘클래스2블록', '영통롯데캐슬엘클래스2BL'),
                ('하남감일한양수자인', '감일수자인')]

for before_name, after_name in excluded_change:
    excluded.loc[excluded['아파트명'] == before_name, '아파트명'] = after_name

**아파트명을 통일시킨 이후 같은 아파트 개수 확인**
* 28개 -> 53개

In [12]:
excluded_list = list(excluded['아파트명'].unique())
exist_list = list(exist['아파트명'].unique())
intersection = set(excluded_list) & set(exist_list) 
len(intersection)

53

**2023년 이전 아파트 중 분양가와 매매가가 있는 아파트만 데이터컬럼으로 추려냄**

In [13]:
for_comparison = excluded[excluded['아파트명'].isin(intersection)]

In [14]:
# 301개 행
for_comparison.shape 

(301, 50)

**아파트명과 공급면적 컬럼을 기준으로 병합**

In [15]:
merged = pd.merge(for_comparison, exist, left_on = ['아파트명', '공급면적'], right_on = ['아파트명', '공급면적'], how='left')
merged.shape

(311, 102)

**'아파트명', '공급면적', '전용면적_x', '전용면적_y' 열을 기준으로 중복된 행들의 공급액 확인**

In [16]:
duplicates = merged[merged.duplicated(subset=['아파트명', '공급면적', '전용면적_x', '전용면적_y'], keep=False)]
duplicates[['아파트명', '공급면적', '전용면적_x', '전용면적_y', '공급액(만원)']]

,아파트명,공급면적,전용면적_x,전용면적_y,공급액(만원)
9,만강(주상복합),75.34,66.48,66.48,38800
10,만강(주상복합),75.34,66.48,66.48,38800
12,만강(주상복합),75.34,66.48,66.48,36400
13,만강(주상복합),75.34,66.48,66.48,36400
34,과천제이드자이,80.73,59.99,59.99,54230
35,과천제이드자이,80.73,59.99,59.99,54230
37,과천제이드자이,80.73,59.99,59.99,52840
38,과천제이드자이,80.73,59.99,59.99,52840
87,e편한세상남양뉴타운,114.95,84.97,84.97,36200
88,e편한세상남양뉴타운,114.95,84.97,84.97,36200


**같은 타입이므로 삭제**  
* '아파트명', '공급면적', '전용면적_x', '전용면적_y', '공급액(만원)' 열을 기준으로 중복된 행을 첫 번째 행을 유지하고 나머지 중복된 행을 제거

In [17]:
merged.drop_duplicates(subset=['아파트명', '공급면적', '전용면적_x', '전용면적_y', '공급액(만원)'], keep='first', inplace=True)

In [18]:
# 삭제되었는지 확인
duplicates = merged[merged.duplicated(subset=['아파트명', '공급면적', '전용면적_x', '전용면적_y'], keep=False)]
duplicates[['아파트명', '공급면적', '전용면적_x', '전용면적_y', '공급액(만원)']]

,아파트명,공급면적,전용면적_x,전용면적_y,공급액(만원)
9,만강(주상복합),75.34,66.48,66.48,38800
12,만강(주상복합),75.34,66.48,66.48,36400
34,과천제이드자이,80.73,59.99,59.99,54230
37,과천제이드자이,80.73,59.99,59.99,52840
87,e편한세상남양뉴타운,114.95,84.97,84.97,36200
90,e편한세상남양뉴타운,114.95,84.97,84.97,35700
209,영통아이파크캐슬3단지,249.77,189.52,189.52,137200
211,영통아이파크캐슬3단지,249.77,189.52,189.52,134700


**그럼에도 병합 후 2행이 추가된 것을 확인**

In [19]:
merged = merged.drop_duplicates()
merged.shape

(303, 102)

In [20]:
# 행 개수가 다른 아파트 확인 -> 영흥숲푸르지오파크비엔

merged_counts = merged['아파트명'].value_counts().reset_index()
for_comparison_counts = for_comparison['아파트명'].value_counts().reset_index()
merged_counts.columns = ['아파트명', '빈도수_merged']
for_comparison_counts.columns = ['아파트명', '빈도수_for_comparison']

frequency_comparison = merged_counts.merge(for_comparison_counts, on='아파트명', how='left')
frequency_comparison[frequency_comparison['빈도수_merged'] != frequency_comparison['빈도수_for_comparison']]

,아파트명,빈도수_merged,빈도수_for_comparison
6,영흥숲푸르지오파크비엔,9,7


**영흥숲푸르지오파크비엔 확인**

In [21]:
odd = merged[(merged['아파트명'] == '영흥숲푸르지오파크비엔') & (merged['면적'].str.contains('109A㎡|109C㎡'))]
odd[['아파트명', '면적', '공급면적', '전용면적_x', '전용면적_y', '공급액(만원)']]

,아파트명,면적,공급면적,전용면적_x,전용면적_y,공급액(만원)
131,영흥숲푸르지오파크비엔,109A㎡,109.96,84.98,84.98,64800
132,영흥숲푸르지오파크비엔,109C㎡,109.96,84.98,84.99,64800
133,영흥숲푸르지오파크비엔,109A㎡,109.96,84.99,84.98,64800
134,영흥숲푸르지오파크비엔,109C㎡,109.96,84.99,84.99,64800


In [22]:
# 영흥숲푸르지오파크비엔에서 전용면적이 다른 행 삭제
merged = merged[~((merged.duplicated(subset=['아파트명', '면적', '공급면적', '공급액(만원)'], keep=False)) 
                  & (merged['전용면적_x'] != merged['전용면적_y']))]

In [23]:
# 제대로 병합되었음을 확인
merged.shape

(301, 102)

In [24]:
# 재확인
merged_counts = merged['아파트명'].value_counts().reset_index()
for_comparison_counts = for_comparison['아파트명'].value_counts().reset_index()
merged_counts.columns = ['아파트명', '빈도수_merged']
for_comparison_counts.columns = ['아파트명', '빈도수_for_comparison']

frequency_comparison = merged_counts.merge(for_comparison_counts, on='아파트명', how='left')
frequency_comparison[frequency_comparison['빈도수_merged'] != frequency_comparison['빈도수_for_comparison']]

,아파트명,빈도수_merged,빈도수_for_comparison


#### **프롬프트 입력을 원활하게 하기 위한 전처리**

우리가 사용할 청약데이터 기준으로 아파트명 다시 변경

In [25]:
original_change = [(b, a) for a, b in excluded_change]
for before_name, after_name in original_change:
    merged.loc[merged['아파트명'] == before_name, '아파트명'] = after_name

컬럼 정리

In [26]:
merged.columns = merged.columns.str.replace('_x', '')
merged = merged.drop(columns=[column_name for column_name in merged.columns if column_name.endswith('_y')])

매매호가 단위 '만원'으로 공급액과 통일

In [27]:
the_lowest = []
the_highest = []

for sales_row in merged['매매호가']:
    if pd.notna(sales_row):
        if "~" in sales_row:
            low, high = sales_row.split("~")
            the_lowest.append(low)
            the_highest.append(high)
        else:
            low = sales_row
            high = None
            the_lowest.append(low)
            the_highest.append(high)
    else:
        the_lowest.append(None)
        the_highest.append(None)

merged['매매최저호가(만원)'] = the_lowest
merged['매매최고호가(만원)'] = the_highest

In [28]:
def cleaning_price(value):
    if pd.notna(value):
        value = value.replace("억", "").replace(",", "").replace(" ", "").strip()
        if len(value) <= 2:
            value += "0000"
    return value

merged['매매최저호가(만원)'] = merged['매매최저호가(만원)'].apply(cleaning_price)
merged['매매최고호가(만원)'] = merged['매매최고호가(만원)'].apply(cleaning_price)

merged.head(1)

,아파트명,법정동주소,위도,경도,세대수,임대세대수,최고층,최저층,최대공급면적,최소공급면적,...,여름관리비,매매호가,전세호가,월세호가,초등학교_학군정보,초등학교_설립정보,초등학교_남학생수,초등학교_여학생수,매매최저호가(만원),매매최고호가(만원)
0,매교역푸르지오SKVIEW(수원),경기도 수원시 팔달구 매교동 209-14번지 일원,37.267318,127.019722,3603,121.0,20.0,7.0,149.85,54.65,...,NaN,12억~14억,"6억 5,000",NaN,NaN,NaN,NaN,NaN,120000,140000


In [29]:
merged.to_csv("for_comparison.csv", encoding='UTF-8', index=False)